<a href="https://colab.research.google.com/github/sonum02/type2diabetes-insight-rag/blob/main/Type2DiabetesAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Get data from recent articles on Type 2 Diabetes in PDF files
!pip install unstructured[pdf]
!pip install -U langchain-community #Installing langchain community
from langchain.document_loaders import DirectoryLoader #Import after the kernel has been restarted

DATA_PATH= "/content/Type2DiabetesPapers"
def load_documents(data_path):
    loader = DirectoryLoader(data_path)
    documents = loader.load()
    return documents
mydocuments = load_documents(DATA_PATH)


  Using cached langchain_community-0.3.14-py3-none-any.whl.metadata (2.9 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached langchain-0.3.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.3.29-py3-none-any.whl.metadata (6.3 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached langchain_community-0.3.14-py3-none-any.whl (2.5 MB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached langchain-0.3.14-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.29-py3-none-any.whl (411 kB)
Using cached pydantic_settings-2.7.1-py3-none-any.whl (29 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langch

In [2]:
#Scrape Mayo website article
import requests
from bs4 import BeautifulSoup
from langchain.schema import Document #Import the document class

# Funtion to get contents of the target website:
def get_website_content(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure successful response
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text()  # Extract text content

#Calling the function to get contents of the website into website_content variable
website_url = "https://www.mayoclinic.org/diseases-conditions/type-2-diabetes/symptoms-causes/syc-20351193"
website_content = get_website_content(website_url)

In [3]:
# Get Diabetes dataset file (Kaggle.json) from Kaggle to Kaggle drive and unzip
!pip install kaggle
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

! mkdir ~/.Kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.Kaggle/
! chmod 600 ~/.Kaggle/kaggle.json
!kaggle datasets download mathchi/diabetes-data-set
! unzip diabetes-data-set.zip

Mounted at /content/drive
Dataset URL: https://www.kaggle.com/datasets/mathchi/diabetes-data-set
License(s): CC0-1.0
  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 16.6MB/s]
Archive:  diabetes-data-set.zip
  inflating: diabetes.csv            


In [4]:
# Load Kaggle dataset and append my docuemnts (recent papers) and Kaggle data
!pip install unstructured
from langchain_community.document_loaders import CSVLoader

DATA_PATH_KaggleData= "/content/diabetes.csv"
def load_documents(data_path_KaggleData):
    loader = CSVLoader(file_path=data_path_KaggleData)
    documents = loader.load()
    return documents

KaggleData = load_documents(DATA_PATH_KaggleData)

# Consolidating data from 3 sources: Recent research papers, Kaggle and Mayo website.
AllData = mydocuments + KaggleData + [Document(page_content=website_content)]

In [5]:
#RAG Retriever
from langchain_text_splitters import RecursiveCharacterTextSplitter

#1: Text Splitter/ Chunking
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)
#2: Splits/ Chunks
chunks= text_splitter.split_documents(AllData)

#Creating embeddings in VectorDB
!pip install chromadb
!pip install tiktoken
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from google.colab import userdata
embeddings = OpenAIEmbeddings(openai_api_key=userdata.get("OpenAIKey"))

#3: Vector Store
VectorDB=Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)

#4: Retriever
retriever=VectorDB.as_retriever()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 26.0 MB/s eta 0:00:0

<ipython-input-5-3d22bccca11b>:20: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=userdata.get("OpenAIKey"))


In [7]:
#LLM
import os
from google.colab import userdata
from langchain.chat_models import ChatOpenAI


# Attempt to get the API key from userdata, fallback to environment variable
openai_api_key = userdata.get('OpenAIKey')
if openai_api_key is None:
    openai_api_key = os.environ.get('OPENAI_API_KEY')
    if openai_api_key is None:
        raise ValueError("Please set either the 'OpenAIKey' in userdata or the 'OPENAI_API_KEY' environment variable.")

#Connect to LLM/ GPT model
model = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")

<ipython-input-7-b067de6bb5bf>:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")


In [8]:
# RAG Chain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [13]:
# RAG chain with Chat history
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain_wHistory = create_stuff_documents_chain(model, qa_prompt)

rag_chain_wHistory = create_retrieval_chain(history_aware_retriever, question_answer_chain_wHistory)

In [24]:
# GUI Gradio
!pip install gradio
import gradio as gr
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

def predict(message, history):
    response = rag_chain_wHistory.invoke({"input": message,"chat_history": chat_history})
    chat_history.extend(
    [
        HumanMessage(content=message),
        AIMessage(content=response["answer"]),
    ]
)
    return response['answer']


demo = gr.ChatInterface(
    predict,
    chatbot=gr.Chatbot(height=200, type="messages"),
    textbox=gr.Textbox(
        placeholder="Ask me anything about Type2 Diabetes",
        container=False,
        scale=7
    ),
    title="Type2 Diabetes Assistant",
    description="I can help answer questions about Type2 Diabetes symptoms, treatment and prevention.",
    theme="soft",
    )

demo.launch(share='True')

/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py:310: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6ac97fc39faa1af9b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
